In [1]:
from typing import Dict, List
import json
from trace import Trace

In [2]:
import requests

JAEGER_TRACES_ENDPOINT = "http://localhost:6686/api/traces?limit=20000&lookback=5m&"
JAEGER_TRACES_SERVICE_PARAM = "service="
JAEGER_TRACES_OPERATION_PARAM = "&operation="

def get_traces(service : str, operation : str):
    """
    Returns list of all traces for a service
    """
    url = JAEGER_TRACES_ENDPOINT + JAEGER_TRACES_SERVICE_PARAM + service + JAEGER_TRACES_OPERATION_PARAM + operation
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.HTTPError as err:
        raise err

    response = json.loads(response.text)
    traces = response["data"]
    return [Trace(t) for t in traces]


In [3]:
run_traces = get_traces("nexus-writer", "Run")
frame_traces = get_traces("digitiser-aggregator", "Frame")
event_formation_traces = get_traces("trace-to-events", "Trace Source Message")

In [4]:
for p in run_traces:
    print(p)
import trace
from importlib import reload
trace = reload(trace)

for t in run_traces:
    t.find_children()

for t in frame_traces:
    t.find_children()

for t in event_formation_traces:
    t.find_children()

for t in run_traces:
    t.find_followed_froms(frame_traces)

for t in frame_traces:
    t.find_followed_froms(event_formation_traces)


4b7b0c35f95241aae6327c8a0ea98b524b7b0c35f95241aae6327c8a0ea98b52, 087e87fb2c850ff3, Run Start Command
4b7b0c35f95241aae6327c8a0ea98b52, 52dc3f83c51002ac, Run Log Data
4b7b0c35f95241aae6327c8a0ea98b52, d32b871ecb0e9245, Run Log Data
4b7b0c35f95241aae6327c8a0ea98b52, 3d5bf0c597a3fd23, Run Log Data
4b7b0c35f95241aae6327c8a0ea98b52, 798ca2178fef747d, Alarm
4b7b0c35f95241aae6327c8a0ea98b52, 763efeaea63ccd87, Sample Environment Log
4b7b0c35f95241aae6327c8a0ea98b52, 1a648d0da9869859, Alarm
4b7b0c35f95241aae6327c8a0ea98b52, 5f1e081f781ea42b, Alarm
4b7b0c35f95241aae6327c8a0ea98b52, 6914288d53e2478a, Alarm
4b7b0c35f95241aae6327c8a0ea98b52, 3831e071f136a40a, Run Stop Command
4b7b0c35f95241aae6327c8a0ea98b52, be4421724e23236d, Frame Event List
4b7b0c35f95241aae6327c8a0ea98b52, 9687e0a9c38db982, Frame Event List
4b7b0c35f95241aae6327c8a0ea98b52, 89132955439b746a, Frame Event List
4b7b0c35f95241aae6327c8a0ea98b52, adfcfe0734cae1a1, Frame Event List
4b7b0c35f95241aae6327c8a0ea98b52, e6ef6ab47fc37991,

In [5]:
runs = [
    Run(s, [s for s in t.spans if s.name == "Frame Event List"])
    for t in run_traces for s in t.spans if s.name == "Run"
]

for r in runs:
    r.print()

NameError: name 'Run' is not defined

In [ ]:
import matplotlib.pyplot as plt




for t in traces:
    t.find_children()
    print(t.validate())
    print(len(t.roots[0].children[0].children[0].children[0].children))

dur = [d for t in traces for d in t.duration_of("find_channel_events")]
#dur = [d for d in dur if d < 500]

plt.hist(dur, bins = 50)
